In [1]:
import findspark
findspark.init()
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from dotenv import load_dotenv

load_dotenv()

# Create a Spark session
spark = SparkSession.builder.appName("MovieRatings").getOrCreate()

def get_rating_data():
    schema = StructType([
        StructField("userId", IntegerType(), True),
        StructField("movieId", IntegerType(), True),
        StructField("rating", FloatType(), True),
        StructField("timestamp", IntegerType(), True)
    ])
    data = spark.read.csv(os.getenv('BASE_PROJECT_PATH') + 'data/u.data', sep='\t', schema=schema, header=False)
    return data

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml. recommendation import ALS
from pyspark.ml. tuning import TrainValidationSplit, ParamGridBuilder

#Load data
movie_ratings = get_rating_data()

#Create test and train set
(training, test) = movie_ratings.randomSplit([0.8, 0.2], seed=42)

#Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
coldStartStrategy="drop", nonnegative = True)

In [4]:
#Tune model using ParamGridBuilder
param_grid = ParamGridBuilder()\
.addGrid(als.rank, [12, 13, 14])\
.addGrid(als.maxIter, [18, 19, 201])\
.addGrid(als.regParam, [.17, .18, .191])\
.build()

#Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
predictionCol="prediction")

#Build cross validation using TrainValidationSplit
tvs = TrainValidationSplit(
estimator=als,
estimatorParamMaps=param_grid,
evaluator=evaluator)

In [5]:
#Fit ALS model to training data
model = tvs.fit(training)
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel
#Generate predictions and evaluate using RMSE
predictions = best_model.transform( test)
rmse = evaluator.evaluate(predictions)
#Print evaluation metrics and model parameters
print ("RMSE = " + str(rmse))
print ("**Best Model**")
print("Rank: "), best_model.rank
print("MaxIter: "), best_model._java_obj.parent().getMaxIter()
print ("RegParam: "), best_model._java_obj.parent().getRegParam()

Py4JJavaError: An error occurred while calling o1428.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 1132.0 failed 1 times, most recent failure: Lost task 4.0 in stage 1132.0 (TID 3082) (LAPTOP-K8C2EPLP executor driver): java.lang.StackOverflowError
	at java.io.ObjectStreamClass.setPrimFieldValues(ObjectStreamClass.java:1399)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2278)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2211)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2287)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2211)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2069)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1573)
	at java.io.ObjectIn